In [1]:
import pymp
from multiprocessing import Manager, cpu_count 
import sys
import os
import glob 
import gezi
from gezi import tqdm
import pandas as pd
import json
import tensorflow as tf
from absl import flags
FLAGS = flags.FLAGS
import melt

from plotly.graph_objs import Scatter,Layout
import plotly
import plotly.offline as py
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots

#setting offilne
plotly.offline.init_notebook_mode(connected=True)
tf.enable_eager_execution()

from sklearn.metrics import roc_auc_score
from projects.feed.listwise.model import Model
from projects.feed.listwise.dataset import Dataset

In [2]:
%%html
<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:10000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>

In [3]:
product='sgsapp'
mark='tuwen'

In [4]:
# 载入模型
model_hour = '2020043019'
valid_hour = '2020043022'
# valid_hour = '2020030812'
model_root = f'/tmp/melt'
# model_root = '/home/gezi/tmp/rank/exps/tuwen/v29/dlrm-norm/'
model_dir = f'{model_root}/{model_hour}'
# model_dir = model_root
tfrecord_dir = f'/home/gezi/tmp/rerank/tfrecords/valid/{valid_hour}/0.record'
# tfrecord_dir = f'{tfrecord_root}/{valid_hour}'
# print('model:', model_dir)
# print('record:', tfrecord_dir)
# argv = open(f'{model_dir}/command.txt').readline().strip().replace('data_version=2', 'data_version=1').split()
# FLAGS(argv)
FLAGS([''])
FLAGS.eager = True
# FLAGS.fields_norm=True
# config.init()
model = Model()
melt.eager.restore(model, model_dir)

In [5]:
dataset = Dataset('valid')

In [6]:
files = gezi.list_files(tfrecord_dir)
total = melt.get_num_records(files) 
total

343335

In [7]:
batch_size = 512
batches = dataset.make_batch(batch_size=batch_size, filenames=files, repeat=False)
num_steps = -int(-total // batch_size)

In [8]:
total = 0
total2 = 0
gauc, gauc_base = 0., 0.
gauc2, gauc2_base = 0., 0.
def deal(y_pred, y_true, positions):
  global total, total2, gauc, gauc2, gauc_base, gauc2_base
  y_pred = y_pred.numpy()
  y_true = y_true.numpy()
  position = positions.numpy()
  for i in range(len(y_pred)):
    len_ = np.sum(position[i] > 0)
#     print(position, len_)
    pred_ = y_pred[i][:len_]
    true_ = y_true[i][:len_]
    position_ = 20 - position[i][:len_]
    if np.sum(true_) > 0 and np.sum(true_ == 0) > 0:
      total2 += 1
      total += len_
      auc = roc_auc_score(true_, pred_)
      auc_base = roc_auc_score(true_, position_)
#       print(true_, pred_, position_)
#       print(auc, auc_base)
      gauc += auc * len_
      gauc_base += auc_base * len_
      gauc2 += auc
      gauc2_base += auc_base      

In [9]:
for x, y in tqdm(batches, total=num_steps):
  deal(tf.sigmoid(model(x)), y, x['positions'])

100%|██████████| 671/671 [06:41<00:00,  1.90it/s]


In [10]:
gauc2 / total2

0.483969879211549

In [11]:
gauc2_base / total2

0.6323128304772208